In [1]:
import os
import sys
import importlib
PARENT_DIR = os.path.realpath('..')+'/'
sys.path.append(PARENT_DIR)
  
import numpy as np
from tqdm import tqdm_notebook
from joblib import Parallel, delayed
import subprocess
from pathlib import Path

try: importlib.reload(sl)
except: import synt_lib as sl

In [2]:
DIRS = sl.get_dirs(parent_dir=PARENT_DIR)
M_PARAMS = sl.get_model_params()

In [17]:
DIRS['RAW_DATA'] = '/data/dudyrev/Speach_Synthesizer/raw_data/'

# Get all mp3 files

In [19]:
dir_ = (DIRS['RAW_DATA']+'rus').replace('//','/')
dir_

'/data/dudyrev/Speach_Synthesizer/raw_data/rus'

In [20]:
hadoop_path = '/user/dudyreve'

In [21]:
def convert_file(path, ext_to='.wav', move_to_hdfs=True, hadoop_path='/user/dudyreve'):
    if isinstance(path, Path):
        path = path.as_posix()
    ext_from = '.'+path.split('.')[-1]
    newpath = path.replace(ext_from, ext_to)
    if os.path.isfile(newpath):
        return 2, newpath
    
    try:
        subprocess.run(['ffmpeg', '-loglevel', 'panic', '-i', path, '-ar', str(M_PARAMS['SAMPLE_RATE']), newpath])
        return 1, newpath
    except Exception as e:
        print(e)
        return 0, newpath
    
    if move_to_hdfs:
        hdfs_path = newpath.replace(dir_,hadoop_path)
        hdfs_dir = '/'.join(hdfs_path.split('/')[:-1])
        subprocess.run(['hdfs','dfs','-mkdir','-p',hdfs_dir])
        subprocess.run(['hdfs','dfs','-copyFromLocal',newpath,hdfs_path])
        subprocess.run(['rm',newpath])
        return 1, hdfs_path
    return 1, newpath

In [22]:
def remove_file(fname):
    if os.path.isfile(fname):
        subprocess.run(['rm',fname])

Remove old wav files

In [23]:
result = Path(dir_).rglob("*.wav")
res_len = len(list(result))
result = Path(dir_).rglob("*.wav")
res_len

0

Convert to new wav files

In [25]:
result = Path(dir_).rglob("*.mp3")
res_len = len(list(result))
result = Path(dir_).rglob("*.mp3")

In [26]:
res_len

1640262

In [27]:
%%time
res_data = Parallel(n_jobs=128, verbose=1)(delayed(convert_file)(path) for path in result);

[Parallel(n_jobs=128)]: Using backend LokyBackend with 128 concurrent workers.
[Parallel(n_jobs=128)]: Done 194 tasks      | elapsed:    4.1s
[Parallel(n_jobs=128)]: Done 544 tasks      | elapsed:    5.9s
[Parallel(n_jobs=128)]: Done 994 tasks      | elapsed:    8.2s
[Parallel(n_jobs=128)]: Done 1544 tasks      | elapsed:   11.2s
[Parallel(n_jobs=128)]: Done 2194 tasks      | elapsed:   14.6s
[Parallel(n_jobs=128)]: Done 2944 tasks      | elapsed:   18.7s
[Parallel(n_jobs=128)]: Done 3794 tasks      | elapsed:   23.3s
[Parallel(n_jobs=128)]: Done 4744 tasks      | elapsed:   28.4s
[Parallel(n_jobs=128)]: Done 5794 tasks      | elapsed:   34.1s
[Parallel(n_jobs=128)]: Done 6944 tasks      | elapsed:   40.3s
[Parallel(n_jobs=128)]: Done 8194 tasks      | elapsed:   47.0s
[Parallel(n_jobs=128)]: Done 9544 tasks      | elapsed:   54.1s
[Parallel(n_jobs=128)]: Done 10994 tasks      | elapsed:  1.0min
[Parallel(n_jobs=128)]: Done 12544 tasks      | elapsed:  1.2min
[Parallel(n_jobs=128)]: Do

[Parallel(n_jobs=128)]: Done 806194 tasks      | elapsed: 67.4min
[Parallel(n_jobs=128)]: Done 818944 tasks      | elapsed: 68.5min
[Parallel(n_jobs=128)]: Done 831794 tasks      | elapsed: 69.6min
[Parallel(n_jobs=128)]: Done 844744 tasks      | elapsed: 70.6min
[Parallel(n_jobs=128)]: Done 857794 tasks      | elapsed: 71.7min
[Parallel(n_jobs=128)]: Done 870944 tasks      | elapsed: 72.8min
[Parallel(n_jobs=128)]: Done 884194 tasks      | elapsed: 73.9min
[Parallel(n_jobs=128)]: Done 897544 tasks      | elapsed: 75.1min
[Parallel(n_jobs=128)]: Done 910994 tasks      | elapsed: 76.2min
[Parallel(n_jobs=128)]: Done 924544 tasks      | elapsed: 77.3min
[Parallel(n_jobs=128)]: Done 938194 tasks      | elapsed: 78.5min
[Parallel(n_jobs=128)]: Done 951944 tasks      | elapsed: 79.6min
[Parallel(n_jobs=128)]: Done 965794 tasks      | elapsed: 80.7min
[Parallel(n_jobs=128)]: Done 979744 tasks      | elapsed: 81.9min
[Parallel(n_jobs=128)]: Done 993794 tasks      | elapsed: 83.1min
[Parallel(

CPU times: user 2h 16min 47s, sys: 8min 18s, total: 2h 25min 6s
Wall time: 2h 19min 12s


[Parallel(n_jobs=128)]: Done 1640262 out of 1640262 | elapsed: 139.2min finished
